# 6개월 집계 데이터 적재
생성한 채널 집계 데이터를 적재 
1000만 세톱 데이터에 대해 차감 연산 실행한 결과  
 - 전체 실행 : 58 초  
 - 차감로직 최초 : 32 초  
 - 차감로직 재실행 : 13 초  


In [1]:
# 필요 라이브러리 임포트  
import socket
import sys
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
from os.path import abspath
import findspark
import time 

In [2]:
# 환경변수 정의  
scale = 1000 # 1000 만 건 수준
PRJ_ROOT = '/user/root'
APP_NAME = f'spark-02-load-inven-6m'
DB_NAME = 'inven'

In [3]:
# 스파크 생성 
def spark_creation():
    spark = SparkSession.builder.master('yarn').appName(APP_NAME)\
    .config('spark.driver.cores', '2').config('spark.driver.memory', '4g')\
    .config('spark.num.executors', '4')\
    .config('spark.executor.cores', '2').config('spark.executor.memory', '2g').getOrCreate()
    sc = spark.sparkContext
    sc
    return spark

In [4]:
spark = spark_creation()

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [5]:
# 데이터의 파일 포맷 및 파일명  
tbl_setop_name = f'inven/table-set-6m-1000'
file_format = 'parquet'
# 샘플 데이터 형식 정의. 읽기/쓰기 편의 제공. 
def define_schema():
    from pyspark.sql.types import StructType, StructField, StringType, LongType, FloatType
    columns = [
        StructField("setop", StringType())
        , StructField("inv_rate_01", FloatType())
        , StructField("inv_val_01", LongType())
        , StructField("inv_req_01", LongType())
        , StructField("inv_rate_02", FloatType())
        , StructField("inv_val_02", LongType())
        , StructField("inv_req_02", LongType())
        , StructField("inv_rate_03", FloatType())
        , StructField("inv_val_03", LongType())
        , StructField("inv_req_03", LongType())
        , StructField("inv_rate_04", FloatType())
        , StructField("inv_val_04", LongType())
        , StructField("inv_req_04", LongType())
        , StructField("inv_rate_05", FloatType())
        , StructField("inv_val_05", LongType())
        , StructField("inv_req_05", LongType())        
        , StructField("inv_rate_06", FloatType())
        , StructField("inv_val_06", LongType())
        , StructField("inv_req_06", LongType())        
    ]
    sample_schema = StructType(columns)
    return sample_schema

In [6]:
# %%time 
# # 기록한 파일 다시 읽어 들이기 
# # 저장 결과 확인하기 
# lines = spark.read.format(file_format).option('path', tbl_setop_name).load()
# data_count = lines.count()
# print(f'DATA Count : {data_count:,}')
# lines.show(5)

In [7]:
# %%time 
# # sparkSQL로 전체 데이터 읽어서 카운트  
# spark.read.format(file_format).load(tbl_setop_name).createOrReplaceTempView('setop_view')
# spark.sql("select count(1) from setop_view").show()

In [8]:
%%time 
# sparkSQL로 전체 데이터 읽어서 카운트  
# 건 수 차이 없는 방식. 최초, 재실행 여부 차이.  
# 최초 : 22 초 , 재실행 : 13 초   
spark.read.format(file_format).load(tbl_setop_name).createOrReplaceTempView('setop_view')
# 원본 데이터에 청약량에 1000 을 강제 할당.  
spark.sql("select * from setop_view").createOrReplaceTempView('setop_org')
# 특정 데이터와 차감할 건수 정보 강제 생성   
# 2백만 
spark.sql("select setop, -1000 inv_req_01, -990 inv_req_02, -980 inv_req_03, -970 inv_req_04, -960 inv_req_05, -950 inv_req_06 from setop_view where setop like 'ST_J%' or setop like 'ST_A%'").createOrReplaceTempView('setop_minus')
# 전체 
#spark.sql("select setop, -1 req from setop_view").createOrReplaceTempView('setop_minus')
# 차감 요청 정보를 원본에 반영  
sql_calc = """
select setop
, sum(inv_req_01) inv_req_01 , sum(inv_req_02) inv_req_02
, sum(inv_req_03) inv_req_03 , sum(inv_req_04) inv_req_04
, sum(inv_req_05) inv_req_05 , sum(inv_req_06) inv_req_06
from 
(select setop
, inv_req_01, inv_req_02, inv_req_03
, inv_req_04, inv_req_05, inv_req_06
from setop_org 
UNION ALL 
select setop
, inv_req_01, inv_req_02, inv_req_03
, inv_req_04, inv_req_05, inv_req_06
from setop_minus
) as A 
group by setop 
"""
spark.sql(sql_calc).createOrReplaceTempView('result')

# 계산 결과와 원본 비교  
spark.sql("select count(1) cnt, sum(inv_req_01+ inv_req_02+ inv_req_03) from setop_org").show()
spark.sql("select count(1) cnt, sum(inv_req_01+ inv_req_02+ inv_req_03) from result").show()

+--------+---------------------------------------------+
|     cnt|sum(((inv_req_01 + inv_req_02) + inv_req_03))|
+--------+---------------------------------------------+
|10000000|                                            0|
+--------+---------------------------------------------+

+--------+---------------------------------------------+
|     cnt|sum(((inv_req_01 + inv_req_02) + inv_req_03))|
+--------+---------------------------------------------+
|10000000|                                  -5940000000|
+--------+---------------------------------------------+

CPU times: user 18.8 ms, sys: 5.45 ms, total: 24.2 ms
Wall time: 33.8 s


In [9]:
spark.stop()